# 설치

In [7]:
!pip install qdrant-client sentence-transformers

In [11]:
!pip install qdrant-client

In [14]:
!pip install python-dotenv

In [8]:
!pip install openai==0.28

   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------------------------------- 76.5/76.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.40.6
    Uninstalling openai-1.40.6:
      Successfully uninstalled openai-1.40.6


# 1. qdrant에 데이터 벡터화하여 저장하기

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

# 환경변수 가져오기
qdrant_api_key = os.getenv("QDRANT_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")

In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
)

print(client.get_collections())

collections=[CollectionDescription(name='meeting_records'), CollectionDescription(name='sample_meeting_rc'), CollectionDescription(name='new_meeting_records'), CollectionDescription(name='ts_meeting_records')]


In [3]:
import pandas as pd
import numpy as np

path = '21대국정조사.csv'
data = pd.read_csv(path)

print(data.head())

    회의번호          발언자                                               발언내용
0  52512  위원장직무대행 박성민  의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적...
1  52512  위원장직무대행 박성민  현재 안건이 미정인 상태로 회의가 열리게 되었기 때문에 추가로 안건을 상정하여 회의...
2  52512  위원장직무대행 박성민  의사일정 제1항 위원장 선임의 건을 상정합니다.   특별위원회 위원장은 국회법 제4...
3  52512       조응천 위원                         우상호 위원을 추천합니다.            
4  52512  위원장직무대행 박성민  또 추천하실 위원님 안 계십니까?    (「없습니다」 하는 위원 있음)  우상호 위...


In [19]:
from sentence_transformers import SentenceTransformer

# 벡터화 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 벡터화
data['vector'] = data['발언내용'].apply(lambda x: model.encode(x))
print("벡터화 완료")

c:\Users\user\.conda\envs\malcam\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\user\.conda\envs\malcam\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


벡터화 완료


In [5]:
# 벡터화 후 파일로 저장
data.to_parquet("벡터화_데이터.parquet")  # 벡터화된 데이터를 저장

In [23]:
# 컬렉션 생성
client.recreate_collection(
    collection_name="meeting_records",
    vectors_config={"size": 384, "distance": "Cosine"}  # 벡터 크기 및 거리 계산 방식
)
print("컬렉션 생성 완료")

C:\Users\user\AppData\Local\Temp\ipykernel_13160\1063533258.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


컬렉션 생성 완료


In [24]:
points = [
    {
        "id": idx,
        "vector": row['vector'],
        "payload": {
            "발언자": row['발언자'],
            "회의번호": row['회의번호'],
            "발언내용": row['발언내용']
        }
    }
    for idx, row in data.iterrows()
]

# 데이터 삽입
batch_size = 20  # 배치 크기 설정
for i in range(0, len(points), batch_size):
    batch = points[i:i + batch_size]
    client.upsert(
        collection_name="meeting_records",
        points=batch
    )
    print(f"배치 {i // batch_size + 1} 삽입 완료")

# 삽입된 데이터 개수 확인
collection_info = client.get_collection("meeting_records")
print(f"총 데이터 개수: {collection_info.vectors_count}")

배치 1 삽입 완료
배치 2 삽입 완료
배치 3 삽입 완료
배치 4 삽입 완료
배치 5 삽입 완료
배치 6 삽입 완료
배치 7 삽입 완료
배치 8 삽입 완료
배치 9 삽입 완료
배치 10 삽입 완료
배치 11 삽입 완료
배치 12 삽입 완료
배치 13 삽입 완료
배치 14 삽입 완료
배치 15 삽입 완료
배치 16 삽입 완료
배치 17 삽입 완료
배치 18 삽입 완료
배치 19 삽입 완료
배치 20 삽입 완료
배치 21 삽입 완료
배치 22 삽입 완료
배치 23 삽입 완료
배치 24 삽입 완료
배치 25 삽입 완료
배치 26 삽입 완료
배치 27 삽입 완료
배치 28 삽입 완료
배치 29 삽입 완료
배치 30 삽입 완료
배치 31 삽입 완료
배치 32 삽입 완료
배치 33 삽입 완료
배치 34 삽입 완료
배치 35 삽입 완료
배치 36 삽입 완료
배치 37 삽입 완료
배치 38 삽입 완료
배치 39 삽입 완료
배치 40 삽입 완료
배치 41 삽입 완료
배치 42 삽입 완료
배치 43 삽입 완료
배치 44 삽입 완료
배치 45 삽입 완료
배치 46 삽입 완료
배치 47 삽입 완료
배치 48 삽입 완료
배치 49 삽입 완료
배치 50 삽입 완료
배치 51 삽입 완료
배치 52 삽입 완료
배치 53 삽입 완료
배치 54 삽입 완료
배치 55 삽입 완료
배치 56 삽입 완료
배치 57 삽입 완료
배치 58 삽입 완료
배치 59 삽입 완료
배치 60 삽입 완료
배치 61 삽입 완료
배치 62 삽입 완료
배치 63 삽입 완료
배치 64 삽입 완료
배치 65 삽입 완료
배치 66 삽입 완료
배치 67 삽입 완료
배치 68 삽입 완료
배치 69 삽입 완료
배치 70 삽입 완료
배치 71 삽입 완료
배치 72 삽입 완료
배치 73 삽입 완료
배치 74 삽입 완료
배치 75 삽입 완료
배치 76 삽입 완료
배치 77 삽입 완료
배치 78 삽입 완료
배치 79 삽입 완료
배치 80 삽입 완료
배치 81 삽입 완료
배치 82 삽입 완료
배치 83 삽입 완료
배치 84 삽입 완료
배

In [25]:
# 삽입된 데이터 개수 확인
collection_info = client.get_collection("meeting_records")
print(f"총 데이터 개수: {collection_info.vectors_count}")

총 데이터 개수: None


In [5]:
results = client.scroll(
    collection_name="sample_meeting_rc",
    offset=None,  # 처음부터 검색
    limit=5  # 상위 5개 결과 반환
)

# 검색 결과 출력
for result in results[0]:  # scroll 반환값은 (points, next_page_offset)의 튜플
    print("ID:", result.id)
    # print("발언내용:", result.payload['발언내용'])
    print("회의번호:", result.payload['회의번호'])
    # print("발언자:", result.payload['발언자'])
    print("요약된발언내용:", result.payload['요약된발언내용'])
    print()

ID: 0
회의번호: 52512
요약된발언내용: 국회에서는 이태원 참사 진상규명과 재발방지를 위한 국정조사특별위원회를 개회하고, 위원장으로 우상호 위원을 선임했습니다. 회의 시작 전, 희생자들을 추모하는 묵념을 진행하였으며, 국정조사의 필요성과 책임에 대해 논의했습니다. 간사로는 더불어민주당의 김교흥 위원과 국민의힘의 이만희 위원이 각각 선임되었습니다. 모든 위원들은 진상 규명과 재발 방지 대책 마련을 위한 협력을 다짐했습니다. 국민의힘은 이태원 참사의 진상 규명과 재발 방지를 위해 국정조사를 진행하겠다고 발표했습니다. 국정조사는 여야 간의 신뢰와 합의에 바탕을 두고 진행할 방침이며, 조사의 목적은 참사의 원인 및 책임 소재 규명과 정부의 재난안전관리 체계 점검입니다. 조사 방법으로는 기관 보고, 서류 제출, 현장 조사, 청문회가 포함됩니다. 조사는 2022년 11월 24일부터 2023년 1월 7일까지 진행되며 필요시 연장 가능하다고 설명했습니다.

전주혜 위원은 국정조사의 정치적 중립성을 우려하며, 조사계획서의 일부 내용에 반대 의견을 제시했습니다. 특히 대통령실 이전과 마약 범죄 단속이 참사의 원인으로 지적된 부분에 대해 반대했고, 대검찰청 포함의 적절성에 의문을 제기했습니다. 그녀는 대검찰청의 조사가 불필요하게 큰 영향을 미칠 수 있다는 우려를 표명했습니다. 이 내용은 이태원 참사와 관련하여 진행되는 국정조사 회의에서의 의견 교환을 담고 있습니다. 김교흥 간사는 대통령실의 용산 이전과 마약범죄가 참사에 미친 영향을 조사해야 한다고 주장하며, 여야 간의 합의를 강조합니다. 이만희 간사는 대검찰청의 증인 채택 문제에 대해 논의하고, 마약 수사 관련 질의 범위를 제한할 필요성을 언급합니다. 조수진 위원은 검수완박 법안으로 인해 마약 수사에 대한 검찰의 수사 권한이 축소된 상황에서의 문제가 있다고 지적하며, 국정조사의 범위에 대해 우려를 표합니다. 전체적으로 국정조사가 정쟁으로 흘러가지 않기를 바라며, 합의를 통해 효과적인 조사 진행을 요청하고 있습니다. 국정조

# 2. 벡터 db 호출하여 발언내용 요약하기

In [32]:
import random
from transformers import pipeline

# Qdrant에서 모든 데이터 불러오기
def fetch_all_data_from_qdrant(collection_name, batch_size=50):
    all_results = []
    offset = None

    while True:
        results, next_page_offset = client.scroll(
            collection_name=collection_name,
            offset=offset,
            limit=batch_size
        )
        all_results.extend(results)

        if not next_page_offset:
            break

        offset = next_page_offset

    return all_results

# 무작위 샘플링
def random_sample(data, sample_size=100):
    return random.sample(data, sample_size)

In [35]:
# BART 요약 모델 로드
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# 텍스트 요약 함수
def summarize_text(text):
    try:
        summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"요약 실패: {e}")
        return "요약 실패"

# 샘플 데이터 요약 실행
def summarize_random_sample(collection_name, sample_size=100):
    # Qdrant에서 모든 데이터 가져오기
    all_data = fetch_all_data_from_qdrant(collection_name)
    print(f"총 데이터 개수: {len(all_data)}")

    # 무작위 샘플링
    sampled_data = random_sample(all_data, sample_size)
    print(f"무작위로 {sample_size}개의 데이터를 샘플링했습니다.")

    # 요약 실행
    summarized_data = []
    for item in sampled_data:
        original_text = item.payload['발언내용']
        summarized_text = summarize_text(original_text)
        summarized_data.append({
            "ID": item.id,
            "원문": original_text,
            "요약": summarized_text
        })
        
    # 결과 저장
    df = pd.DataFrame(summarized_data)
    output_path = "qdrant_요약_샘플_결과.csv"
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"요약된 샘플 데이터가 '{output_path}'에 저장되었습니다.")

# 실행
summarize_random_sample("meeting_records", sample_size=100)

총 데이터 개수: 9682
무작위로 100개의 데이터를 샘플링했습니다.
요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 47. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 150, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 150, but your input_length is only 63. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=31)
Your max_length is set to 150, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your 

요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 150, but your input_length is only 49. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Your max_length is set to 150, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 150, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your 

요약 실패: index out of range in self


...

요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 150, but your input_length is only 68. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)
Your max_length is set to 150, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 150, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your

요약 실패: index out of range in self


Your max_length is set to 150, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 150, but your input_length is only 112. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Your max_length is set to 150, but your input_length is only 71. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)
Your max_length is set to 150, but your input_length is only 33. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
You

요약된 샘플 데이터가 'qdrant_요약_샘플_결과.csv'에 저장되었습니다.


In [36]:
sameple = pd.read_csv('qdrant_요약_샘플_결과.csv')
sameple.head()


,ID,원문,요약
0,5914,"저는 진짜 이해할 수가 없는데 그러면 결국은, 용산소방서장의 인파 통제 요청이 23...",요약 실패
1,2347,"아니, 우리가 현장조사 나갔을 때는 지금 말씀하신 취지로 보고 하셨잖아요. 그러니까...","‘ ‘”’ ‘’, “” ””, “ ’’ “, ”, ’”. “.” “ ..."
2,4513,김광호 증인 말씀하십시오.,"김광호 증인 “십’s” ‘‘’ “’’ ”“” ’” ”, “ ’,’..."
3,2024,"예, 그렇습니다.","“I’m sorry, I’ve got to go.” “What’s w..."
4,4414,"이것은 진짜 너무들 해요, 정말 너무들 해.","넉 ‘’ ‘”’ “’’, ’”, ‘, ’, ”, “,” ..."


In [27]:
all_data = fetch_all_data_from_qdrant("meeting_records")
print(f"불러온 데이터 개수: {len(all_data)}")

불러온 데이터 개수: 9682


In [28]:
for item in all_data[:5]:  # 샘플 데이터 5개만 출력
    print(item.payload)

{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적위원 4분의 1 이상의 회의 개회 요구가 있었습니다. 이에 제400회 국회(정기회) 제1차 용산 이태원 참사 진상규명과 재발방지를 위한 국정조사특별위원회를 개회하겠습니다.  국회법 제47조제2항에 따라 위원장 선임 시까지 위원장 직무를 대행하게 되었습니다.            '}
{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '현재 안건이 미정인 상태로 회의가 열리게 되었기 때문에 추가로 안건을 상정하여 회의를 진행하고자 합니다.   위원님 여러분 이의가 없습니까?     (「예」 하는 위원 있음)  이의가 없으시면 의사일정을 변경하여 안건을 상정하도록 하겠습니다.            '}
{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '의사일정 제1항 위원장 선임의 건을 상정합니다.   특별위원회 위원장은 국회법 제47조제1항에 따라 특별위원회에서 호선하고 본회의에 보고하도록 되어 있습니다.   위원장 선임 방법은 위원님들께서 특별한 의견이 없으시면 그간의 특별위원회 관례에 따라 구두 추천에 의하여 선임하고자 합니다.  위원님 여러분 이의가 없으십니까?    (「예」 하는 위원 있음)  그러면 위원님들께서 이번 국정조사를 원만하고 공정하게 이끌어 주실 가장 적임자라고 생각하시는 위원님을 우리 위원회 위원장으로 추천해 주시기 바랍니다.  조응천 위원님.            '}
{'발언자': '조응천 위원', '회의번호': 52512, '발언내용': '우상호 위원을 추천합니다.            '}
{'발언자': '위원장직무대행 박성민', '회의번호': 52512, '발언내용': '또 추천하실 위원님 안 계십니까?    (「없습니다」 하는 위원 있음)  우상호 위원님을 위원장으로 선임하고자 합니다.  위원님 여러분 이의 

# 3. 요약 후 벡터화 작업 

In [1]:
import pandas as pd
import numpy as np

path = '21대국정조사.csv'
data = pd.read_csv(path)

print(data.head())

    회의번호          발언자                                               발언내용
0  52512  위원장직무대행 박성민  의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적...
1  52512  위원장직무대행 박성민  현재 안건이 미정인 상태로 회의가 열리게 되었기 때문에 추가로 안건을 상정하여 회의...
2  52512  위원장직무대행 박성민  의사일정 제1항 위원장 선임의 건을 상정합니다.   특별위원회 위원장은 국회법 제4...
3  52512       조응천 위원                         우상호 위원을 추천합니다.            
4  52512  위원장직무대행 박성민  또 추천하실 위원님 안 계십니까?    (「없습니다」 하는 위원 있음)  우상호 위...


In [3]:
# 회의번호별로 발언내용 합치기
grouped_data = data.groupby("회의번호")["발언내용"].apply(lambda x: " ".join(x)).reset_index()

# 결과 확인
print(grouped_data.head())

    회의번호                                               발언내용
0  52512  의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적...
1  52622  의석을 정돈해 주시기 바랍니다.  성원이 되었으므로 제401회 국회(임시회) 제2차...
2  52629  의석을 정돈해 주시기 바랍니다.  지금부터 헌법 제61조, 국회법 제127조와 국정...
3  52638  의석을 정돈해 주시기 바랍니다.  지금부터 헌법 제61조, 국회법 제127조와 국정...
4  52643  의석을 정돈해 주시기 바랍니다.  지금부터 헌법 제61조, 국회법 제127조와 국정...


In [4]:
# 합쳐진 데이터 저장
grouped_data.to_csv("회의번호별_발언내용_합침.csv", index=False, encoding="utf-8-sig")
print("회의번호별로 발언내용을 합친 파일이 저장되었습니다.")

회의번호별로 발언내용을 합친 파일이 저장되었습니다.


In [4]:
import openai
import pandas as pd
from dotenv import load_dotenv
import os
import re

# 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# 특수 문자 제거
def clean_text(text):
    return text.encode("utf-8", "ignore").decode("utf-8")

grouped_data["발언내용"] = grouped_data["발언내용"].apply(clean_text)

# 텍스트 길이 제한
def truncate_text(text, max_length=4000):
    return text[:max_length] if len(text) > max_length else text

grouped_data["발언내용"] = grouped_data["발언내용"].apply(truncate_text)

# 요약 함수
def gpt4_turbo_summarize(text):
    try:
        response = openai.Completion.create(
            engine="gpt-4-turbo",
            prompt=f"다음 텍스트를 요약해 주세요:\n{text}",
            max_tokens=150,
            temperature=0.5
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"요약 실패: {e}")
        return "요약 실패"

# 데이터 요약
grouped_data["요약"] = grouped_data["발언내용"].apply(gpt4_turbo_summarize)

# 저장 시 UTF-8 지정
grouped_data.to_csv("회의번호별_발언내용_요약.csv", index=False, encoding="utf-8-sig")

요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)


In [2]:
# 데이터 로드 및 처리
file_path = "회의번호별_발언내용_합침.csv"
grouped_data = pd.read_csv(file_path)

In [3]:
grouped_data

,회의번호,발언내용
0,52512,의석을 정돈해 주시기 바랍니다. 국회법 제52조 및 제3호에 따라 특별위원회 재적...
1,52622,의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제401회 국회(임시회) 제2차...
2,52629,"의석을 정돈해 주시기 바랍니다. 지금부터 헌법 제61조, 국회법 제127조와 국정..."
3,52638,"의석을 정돈해 주시기 바랍니다. 지금부터 헌법 제61조, 국회법 제127조와 국정..."
4,52643,"의석을 정돈해 주시기 바랍니다. 지금부터 헌법 제61조, 국회법 제127조와 국정..."
5,52644,"의석을 정돈해 주시기 바랍니다. 지금부터 헌법 제61조, 국회법 제127조와 국정..."
6,52647,"의석을 정돈하여 주시기 바랍니다. 지금부터 헌법 제61조, 국회법 제127조와 국..."
7,52650,"의석을 정돈해 주시기 바랍니다. 지금부터 헌법 제61조, 국회법 제127조와 국..."
8,52653,"의석을 정돈해 주시기 바랍니다. 지금부터 헌법 제61조, 국회법 제127조와 국..."


In [16]:
# 발언내용 전처리
grouped_data["발언내용"].fillna("", inplace=True)
grouped_data["발언내용"] = grouped_data["발언내용"].apply(clean_text)

# 요약 수행
grouped_data["요약"] = grouped_data["발언내용"].apply(summarize_long_text)

# 데이터 저장
grouped_data.to_csv("회의번호별_발언내용_요약.csv", index=False, encoding="utf-8-sig")
print("요약된 데이터가 저장되었습니다.")

C:\Users\user\AppData\Local\Temp\ipykernel_50232\3244488822.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  grouped_data["발언내용"].fillna("", inplace=True)


요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't e

# 4. txt파일로 해서 작업

In [21]:
import pandas as pd

# CSV 파일 로드
csv_file = "21대국정조사.csv"
data = pd.read_csv(csv_file, encoding='utf-8')

# 필요한 열만 추출
text_data = data[['회의번호', '발언내용']]

# 텍스트 파일로 저장
with open("21대국정조사.txt", "w", encoding="utf-8") as file:
    for index, row in text_data.iterrows():
        file.write(f"회의번호: {row['회의번호']}\n발언내용: {row['발언내용']}\n\n")

print("텍스트 파일로 저장 완료: 21대국정조사.txt")

텍스트 파일로 저장 완료: 21대국정조사.txt


In [22]:
import re

def clean_text(text):
    """
    텍스트에서 문제를 일으킬 수 있는 특수문자 제거 또는 대체
    """
    if not isinstance(text, str):
        return ""
    text = text.replace("\u2014", "-")  # 긴 대시를 일반 대시로
    text = text.replace("\u2013", "-")  # en-dash 대체
    text = text.replace("\u2026", "...")  # 말줄임표 대체
    text = text.replace("’", "'").replace("“", '"').replace("”", '"')  # 일반 문자 대체
    text = re.sub(r"[^\x00-\x7F]+", " ", text)  # 비ASCII 문자 제거
    return text.strip()

In [24]:
import re

def clean_text(text):
    """
    텍스트에서 문제를 일으키는 특수문자 제거 또는 대체
    """
    if not isinstance(text, str):  # 텍스트가 아닌 경우 처리
        return ""
    text = text.replace("\u2014", "-")  # 긴 대시를 일반 대시로 대체
    text = text.replace("\u2013", "-")  # en-dash 대체
    text = text.replace("\u2026", "...")  # 말줄임표 대체
    text = re.sub(r"[^\w\s가-힣.,!?\"\'():;<>%-]", " ", text)  # 한글, 공백, 기본 특수문자만 허용
    text = re.sub(r"\s+", " ", text)  # 연속된 공백을 하나로
    return text.strip()

In [25]:
# 텍스트 파일 읽기
with open("21대국정조사.txt", "r", encoding="utf-8") as file:
    raw_text = file.readlines()

# 텍스트 정리
cleaned_text = [clean_text(line) for line in raw_text]

# 정리된 텍스트를 새로운 파일에 저장
with open("21대국정조사_정리됨.txt", "w", encoding="utf-8") as file:
    file.writelines(cleaned_text)

print("정리된 텍스트 저장 완료: 21대국정조사_정리됨.txt")

정리된 텍스트 저장 완료: 21대국정조사_정리됨.txt


In [26]:
def split_text(text, max_length=1000):
    """
    긴 텍스트를 GPT 요약 가능하도록 분할
    """
    text = text.strip()
    return [text[i:i + max_length] for i in range(0, len(text), max_length)]

In [ ]:
import openai

# 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

def gpt4_summarize(text):
    """
    GPT-4를 사용하여 텍스트 요약
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for summarizing text."},
                {"role": "user", "content": f"다음 텍스트를 요약해 주세요:\n{text}"}
            ],
            max_tokens=150,
            temperature=0.5
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"요약 실패: {e}")
        return "요약 실패"


In [28]:
# 정리된 텍스트 불러오기
with open("21대국정조사_정리됨.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# 요약 수행
summaries = []
for line in lines:
    if line.startswith("발언내용:"):
        content = line.replace("발언내용:", "").strip()
        chunks = split_text(content)  # 텍스트 분할
        summary = " ".join([gpt4_summarize(chunk) for chunk in chunks])  # 분할된 텍스트 각각 요약
        summaries.append(summary)

print("요약 작업 완료.")


요약 작업 완료.


In [29]:
# 회의번호와 요약 텍스트를 데이터프레임으로 생성
result_data = pd.DataFrame({
    "회의번호": data['회의번호'],
    "요약된발언내용": summaries
})

# CSV로 저장
result_data.to_csv("21대국정조사_요약결과.csv", index=False, encoding="utf-8-sig")
print("요약 결과 CSV 저장 완료: 21대국정조사_요약결과.csv")

ValueError: array length 0 does not match index length 9682

In [ ]:
import pandas as pd
import re

# 1. 텍스트 파일 읽기
with open("21대국정조사_정리됨.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# 2. 회의번호와 발언내용 분리 함수
def parse_lines(lines):
    records = []
    for line in lines:
        match = re.match(r"회의번호: (\d+).*발언내용: (.+)", line.strip())
        if match:
            meeting_number = match.group(1)  # 회의번호
            speech = match.group(2)  # 발언내용
            records.append({"회의번호": meeting_number, "발언내용": speech})
    return records

# 3. 데이터프레임 생성
parsed_data = parse_lines(lines)
data = pd.DataFrame(parsed_data)

# 결과 확인
print(data.head())


    회의번호                                               발언내용
0  52512  의사일정 추가하는 변경안은 국회법 제77조에 따라 위원장이 간사와 협의하여 정하도록...


In [32]:
# 요약 함수 (GPT-4 사용 예시)
def gpt4_summarize(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"다음 텍스트를 요약해 주세요:\n{text}"}
            ],
            max_tokens=150,
            temperature=0.5
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"요약 실패: {e}")
        return "요약 실패"

# 요약 작업
data["요약된발언내용"] = data["발언내용"].apply(gpt4_summarize)

# 결과 저장
data.to_csv("21대국정조사_요약결과.csv", index=False, encoding="utf-8-sig")
print("요약 완료 및 저장: 21대국정조사_요약결과.csv")

요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 완료 및 저장: 21대국정조사_요약결과.csv


In [20]:
# 요약 수행 및 결과 저장
data['발언내용_요약'] = data['발언내용_정리'].apply(gpt4_summarize)

# 원본, 정리된 텍스트, 요약본 확인
print(data[['발언내용', '발언내용_정리', '발언내용_요약']].head())

요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
요약 실패: 'latin-1' codec can't encode character '\u2014' in position 76: ordinal not in range(256)
                                                발언내용  \
0  의석을 정돈해 주시기 바랍니다.  국회법 제52조 및 제3호에 따라 특별위원회 재적...   
1  의석을 정돈해 주시기 

# 5. 2차 요약작업

In [3]:
import pandas as pd
import os

# CSV 파일 읽기
data = pd.read_csv("21대국정조사.csv", encoding="utf-8-sig")

# 출력 폴더 생성
output_dir = "회의별_txt"
os.makedirs(output_dir, exist_ok=True)

# 회의번호별로 텍스트 파일 저장
for 회의번호, group in data.groupby("회의번호"):
    file_path = os.path.join(output_dir, f"{회의번호}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        for 내용 in group["발언내용"]:
            f.write(내용 + "\n")
print(f"{output_dir} 폴더에 회의번호별 txt 파일이 저장되었습니다.")


회의별_txt 폴더에 회의번호별 txt 파일이 저장되었습니다.


In [4]:
import re

# 텍스트 정리 함수
def clean_text(text):
    """
    텍스트에서 문제를 일으키는 특수문자 제거 또는 대체
    """
    if not isinstance(text, str):  # 텍스트가 아닌 경우 처리
        return ""
    text = text.replace("\u2014", "-")  # 긴 대시를 일반 대시로 대체
    text = text.replace("\u2013", "-")  # en-dash 대체
    text = text.replace("\u2026", "...")  # 말줄임표 대체
    text = re.sub(r"[^\w\s가-힣.,!?\"\'():;<>%-]", " ", text)  # 한글, 공백, 기본 특수문자만 허용
    text = re.sub(r"\s+", " ", text)  # 연속된 공백을 하나로
    return text.strip()

# 텍스트 파일을 정리
for txt_file in os.listdir(output_dir):
    file_path = os.path.join(output_dir, txt_file)
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    cleaned_lines = [clean_text(line) for line in lines]
    
    # 정리된 텍스트 다시 저장
    with open(file_path, "w", encoding="utf-8") as f:
        f.writelines(line + "\n" for line in cleaned_lines)
print("텍스트 정리 완료.")


텍스트 정리 완료.


In [13]:
import openai
from dotenv import load_dotenv
import os

# 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# 요약 함수
def summarize_text_gpt4(content, max_length=4000, chunk_size=3000):
    """
    GPT-4o-mini를 사용하여 텍스트 요약.
    - max_length: API 호출 시 텍스트 최대 길이.
    - chunk_size: 텍스트를 분할 처리할 길이.
    """
    summaries = []
    try:
        # 텍스트를 chunk_size 단위로 분할
        chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
        for chunk in chunks:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that summarizes text in Korean."},
                    {"role": "user", "content": f"Summarize the following text:\n\n{chunk}"}
                ],
                max_tokens=1000
            )
            summaries.append(response['choices'][0]['message']['content'].strip())
        return " ".join(summaries)
    except Exception as e:
        return f"요약 실패: {str(e)}"

# 요약 수행
summaries = {}
log_errors = []

for txt_file in os.listdir(output_dir):
    file_path = os.path.join(output_dir, txt_file)
    
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    
    # 텍스트 길이 초과 처리
    if len(content) > 4000:
        print(f"파일 {txt_file} 텍스트 길이가 길어 분할 작업 수행.")
    
    # 요약 수행
    try:
        summary = summarize_text_gpt4(content)
    except Exception as e:
        summary = f"요약 실패: {str(e)}"
        log_errors.append({"file": txt_file, "error": str(e)})
    
    # 회의번호별 요약 저장
    회의번호 = os.path.splitext(txt_file)[0]
    summaries[회의번호] = summary

# 오류 로그 저장
if log_errors:
    with open("요약_오류_로그.txt", "w", encoding="utf-8") as f:
        for log in log_errors:
            f.write(f"파일: {log['file']}, 오류: {log['error']}\n")

print("요약 작업 완료. 오류 로그는 '요약_오류_로그.txt'에 저장되었습니다.")

파일 52512.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52622.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52629.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52638.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52643.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52644.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52647.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52650.txt 텍스트 길이가 길어 분할 작업 수행.
파일 52653.txt 텍스트 길이가 길어 분할 작업 수행.
요약 작업 완료. 오류 로그는 '요약_오류_로그.txt'에 저장되었습니다.


In [14]:
import pandas as pd

# 요약 결과를 데이터프레임으로 변환
result_data = pd.DataFrame({
    "회의번호": list(summaries.keys()),
    "요약된발언내용": list(summaries.values())
})

# 요약 결과를 CSV 파일로 저장
result_csv_path = "21대국정조사_GPT4_요약결과.csv"
result_data.to_csv(result_csv_path, index=False, encoding="utf-8-sig")

print(f"요약 결과가 CSV 파일로 저장되었습니다: {result_csv_path}")

요약 결과가 CSV 파일로 저장되었습니다: 21대국정조사_GPT4_요약결과.csv


# 6. 요약된 내용 qdrant에 저장

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

# 환경변수 가져오기
qdrant_api_key = os.getenv("QDRANT_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")

In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
)

print(client.get_collections())

collections=[CollectionDescription(name='meeting_records'), CollectionDescription(name='ts_meeting_records')]


In [3]:
import pandas as pd
import numpy as np

path = '21대국정조사_GPT4_요약결과.csv'
data = pd.read_csv(path)

print(data.head())

    회의번호                                            요약된발언내용
0  52512  국회에서는 이태원 참사 진상규명과 재발방지를 위한 국정조사특별위원회를 개회하고, 위...
1  52622  제401회 국회(임시회) 제2차 용산 이태원 참사 진상규명과 재발방지를 위한 국정조...
2  52629  헌법 제61조와 국회법 제127조에 따라 제401회 국회(임시회) 이태원 참사 진상...
3  52638  국회에서 이태원 참사 진상규명 및 재발 방지를 위한 국정조사특별위원회가 열렸습니다....
4  52643  국정조사특별위원회가 이태원 참사와 관련한 청문회를 시작하였습니다. 위원장은 조사 대...


In [5]:
from sentence_transformers import SentenceTransformer

# 벡터화 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 벡터화
data['vector'] = data['요약된발언내용'].apply(lambda x: model.encode(x))
print("벡터화 완료")

벡터화 완료


In [6]:
# 컬렉션 생성
client.recreate_collection(
    collection_name="sample_meeting_rc",
    vectors_config={"size": 384, "distance": "Cosine"}  # 벡터 크기 및 거리 계산 방식
)
print("컬렉션 생성 완료")

C:\Users\user\AppData\Local\Temp\ipykernel_27708\1562673504.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


컬렉션 생성 완료


In [10]:
# 데이터 정리: Qdrant에 삽입할 포맷으로 준비
qdrant_payload = [
    {
        "id": index,  # index를 정수로 사용
        "vector": vector.tolist(),
        "payload": {
            "회의번호": str(data.loc[index, "회의번호"]),
            "요약된발언내용": data.loc[index, "요약된발언내용"]
        }
    }
    for index, vector in enumerate(data['vector'])
]

print(f"Qdrant에 삽입할 데이터 {len(qdrant_payload)}건 준비 완료.")

Qdrant에 삽입할 데이터 9건 준비 완료.


In [11]:
# Qdrant에 데이터 삽입
client.upsert(
    collection_name="sample_meeting_rc",
    points=qdrant_payload
)

print("데이터 삽입 완료.")

데이터 삽입 완료.
